In [ ]:
#Import everything from biocrnpyler
!pip install biocrnpyler[all]
!pip install pandas
from biocrnpyler import *

#Used for simulations via Bioscrape
# Install emcee
!pip install emcee
 
# Install bioscrape
!git clone https://github.com/biocircuits/bioscrape.git
!(cd bioscrape; python setup.py install)
 
# Make sure it worked
import bioscrape


     |████████████████████████████████| 112kB 6.3MB/s 
     |████████████████████████████████| 6.2MB 8.2MB/s 
     |████████████████████████████████| 51kB 3.2MB/s 
Cloning into 'bioscrape'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 1724 (delta 3), reused 1 (delta 0), pack-reused 1715
Receiving objects: 100% (1724/1724), 134.71 MiB | 22.48 MiB/s, done.
Resolving deltas: 100% (1090/1090), done.
Installing Bioscrape...
Compiling bioscrape/random.pyx because it changed.
Compiling bioscrape/types.pyx because it changed.
Compiling bioscrape/simulator.pyx because it changed.
Compiling bioscrape/inference.pyx because it changed.
[1/4] Cythonizing bioscrape/inference.pyx
[2/4] Cythonizing bioscrape/random.pyx
[3/4] Cythonizing bioscrape/simulator.pyx
[4/4] Cythonizing bioscrape/types.pyx
Bioscrape Cythonized.
Installing Lineage...
Compiling lineage/lineage.pyx because it changed.
[1/1] Cythoni

In [ ]:
# Create Param Dict
kb, ku, ktx, ktl, kdeg, cooperativity = 100, 10, 3, 2, 1, 1
kb_dcas_grna, ku_dcas_rgna = 100, .01  # binding constants for dCas9 and Guide RNAs
kb_dcas_dna, ku_dcas_dna = 100, .1
parameters = {"kb": kb, "ku": ku, "ktx": ktx, "ktl": ktl, "kdeg": kdeg, "cooperativity": cooperativity,
              # default params
              ("dCas9_binding", "kb"): kb_dcas_grna, ("dCas9_binding", "ku"): ku_dcas_rgna,
              # binding constants for gRNA and dCas9
              ("dCas9_dna_binding", "kb"): kb_dcas_dna, ("dCas9_dna_binding", "ku"): ku_dcas_dna,
              # binding constants for DNA and dcas-guide complex
              }

In [ ]:
# Create an assembly to express dCas9
const_dCas_assembly = DNAassembly("dCas9", promoter="P", rbs="BCD")
# Get the dCas Species. Note: This could also be defined above and passed into the assembly as protein=...
dCas = const_dCas_assembly.protein 
#changed _protein to protein


# Create a Guide RNA Guide1
#gRNA = guideRNA("guide1", dCas=dCas)
gRNA = RNA("guide1", dCas=dCas)
#issue: guideRNA is untracked in github, may not be part of the documentation
#instead, look at the documentation for DNA assembly to find what a transcript needs to be

# Create an assembly to express the guide RNA
const_gRNA_assembly = DNAassembly("gRNA", transcript=gRNA, promoter="P", rbs=None)
#from documentation: If transcript is None and protein is not None, the DNAassembly will use its transcription mechanisms to produce the protein.

TypeError: ignored

In [ ]:
class RNA(Component):
    """A class to represent Components made of RNA. Produces no reactions."""

    def __init__(self, name: str, length=0, dCas9=None, attributes=None, **keywords):
        """Initialize a RNA object to store RNA related information
        :param name: name of the rna
        :param length: number of basepairs (int)
        :param dCas9: complementary dCas9 to RNA 
        :param attributes: Species attribute
        :param keywords: pass into the parent's (Component) initializer
        """

        self.length = length
        self.species = self.set_species(name, material_type="rna",
                                        attributes=attributes)
        Component.__init__(self=self, name=name, **keywords)
        self.get_dCas9Complex(dCas9)

    def get_species(self) -> Species:
        return self.species

    def update_species(self) -> List[Species]:
        species = [self.get_species()]
        return species

    def update_reactions(self) -> List:
        return []

  
    def get_dCas9Complex(self) -> List[Species]:
        if self.dCas9 is not None:
            species = [Complex([self.RNA, self.dCas9])]

In [ ]:
# Get the guideRNA:dCas9 complex
repressor = gRNA.get_dCasComplex()
#repressor=Complex([gRNA,dCas9_dna_binding])
# set the unbinding  rate of the repressor dna complex with rnap to be very high (preventing transcription)
parameters[("transcription", repressor.name, "ku")] = 10000
# set the RNAP transcription rate lower when gRNA:dCAS9 is bound
parameters[("transcription", repressor.name, "ktx")] = .1

# Create a Promoter regulated by the repressor
P_reg = RegulatedPromoter("P_regulated", regulators=[repressor], leak=True)
P_reg.default_mechanisms['binding'] = Reversible_Bimolecular_Binding("dCas9_dna_binding")
# Create an assembly with the regulated promoter
reg_assembly = DNAassembly(name="reporter", promoter=P_reg, rbs="BCD")

NameError: ignored

In [ ]:
# Create a list of components to add to the mixture (these could also be added one-by-one with Mixture.add_component(...)
components = [const_dCas_assembly, const_gRNA_assembly, gRNA, reg_assembly]

# Create a BasicExtract Mixture
reaction_mix = BasicExtract("txtl", components=components, parameters=parameters)

# Compile a CRN
CRN = reaction_mix.compile_crn()
print(repr(CRN))

file_name = "dcas9_repression_test.xml"
f = CRN.write_sbml_file(file_name)

NameError: ignored

In [ ]:
# Initial Condition Dict: repr(specie) --> concentration. Default is 0
# Note in SBML all the species names have ":" replaced with "_".
x0_no_dcas = {repr(const_dCas_assembly.dna): 0, repr(reg_assembly.dna): 1, repr(const_gRNA_assembly.dna): 10,
              "complex_Ribo": 20, "protein_RNAP": 10, "protein_RNAase": 10}
x0_with_dcas = {repr(const_dCas_assembly.dna): 2., repr(reg_assembly.dna): 1, repr(const_gRNA_assembly.dna): 10,
                "complex_Ribo": 20, "protein_RNAP": 10, "protein_RNAase": 10}

In [ ]:
#Bioscrape Simulation
import numpy as np
import pylab as plt

In [ ]:
timepoints = np.arange(0, 100, .01)
print("Simulating")
sim_no_cas, model= CRN.simulate_with_bioscrape_deterministic(timepoints, f, x0_no_dcas)
sim_with_cas, model= CRN.simulate_with_bioscrape_deterministic(timepoints, f, x0_with_dcas)

rep_ind = model.get_species_index("protein_reporter")
dcas_ind = model.get_species_index(repr(dCas))
grna_ind = model.get_species_index(repr(gRNA))
grep_ind = model.get_species_index(repr(reg_assembly.dna))
grep_rep_ind = model.get_species_index("complex_complex_1x_rna_guide1_protein_dCas9_dna_reporter")


plt.figure(figsize = (10, 10))
plt.subplot(311)
plt.title("Protein Products")
plt.plot(timepoints, sim_no_cas.py_get_result()[: , rep_ind], color = "red", label = "Reporter: No dCas9")
plt.plot(timepoints, sim_with_cas.py_get_result()[: , rep_ind],":", color = "red", label = "Reporter: With dCas9")
plt.plot(timepoints, sim_no_cas.py_get_result()[: , dcas_ind], color = "blue", label = "dCas9: No dCas9")
plt.plot(timepoints, sim_with_cas.py_get_result()[: , dcas_ind],":", color = "blue", label = "dCas9: With dCas9")
plt.legend()

plt.subplot(312)
plt.title("RNAs")
plt.plot(timepoints, sim_no_cas.py_get_result()[: , grna_ind], color = "cyan", label = "gRNA: No dCas9")
plt.plot(timepoints, sim_with_cas.py_get_result()[: , grna_ind],":", color = "cyan", label = "gRNA: With dCas9")
plt.legend()

plt.subplot(313)
plt.plot(timepoints, sim_no_cas.py_get_result()[: , grep_ind], color = "red", label = "dna_reporter: No dCas9")
plt.plot(timepoints, sim_with_cas.py_get_result()[: , grep_ind],":", color = "red", label = "dna_reporter: With dCas9")
plt.plot(timepoints, sim_no_cas.py_get_result()[: , grep_rep_ind], color = "blue", label = "repressed dna_reporter: No dCas9")
plt.plot(timepoints, sim_with_cas.py_get_result()[: , grep_rep_ind],":", color = "blue", label = "repressed dna_reporter: With dCas9")

plt.plot()
plt.legend()
plt.show()